# WORKER–VALIDATOR | Agentic Pattern

A two-agent pattern that introduces a quality control loop. A Worker agent attempts the task, and a Validator agent reviews the result against the original instructions. If the result is inadequate, the Validator provides feedback, and the Worker revises the output. This loop continues until the Validator approves the result or a termination condition is reached.

## Problem Addressed
This pattern improves the reliability and quality of output for tasks that may require iterative refinement, quality assurance, or structured reasoning. It is especially useful when the cost of error is high or task success criteria are non-trivial to evaluate.

## Pattern Structure

**Agents |**
- `Worker`: Interprets the task and produces an initial output.
- `Validator`: Evaluates whether the Worker’s output satisfies the task requirements, and provides structured feedback if not.

**Coordination Topology |**
- Sequential feedback loop.
- `Caller → Worker → Validator → (loop: Worker ← Validator)` until the Validator approves the result or a max iteration limit is reached.

## Assumptions

Worker:
- Has all the assumptions of the Worker pattern (tools, reasoning, etc.).
- Can **interpret and apply feedback** to revise prior outputs.

Validator:
- Possesses the capability to **interpret task requirements**, **evaluate the Worker’s output**, and **generate feedback or approval**.
- Can compare against a reference prompt or desired criteria.

Agents may or may not share memory. Coordination state must persist across iterations.

## Inputs
- A task description (e.g., instructions or goals).
- Optionally: a rubric or evaluation criteria for the Validator.

## Outputs
- A final validated result.
- Optionally: Validator’s rationale or evaluation score.

## Behavioural Flow

1. Input task is passed to the Worker.
2. Worker generates initial output.
3. Output and task are passed to the Validator.
4. Validator evaluates the output:
    - If acceptable, the result is returned.
    - If not, Validator returns structured feedback.
5. Worker revises output based on feedback.
6. Loop returns to step 3. Repeat until approval or max iterations reached.

## Strengths
- Higher output quality and reliability.
- Supports more complex, ambiguous, or high-stakes tasks.
- Validator role introduces an externalized standard of correctness or fit.

## Weaknesses
- Higher latency and compute cost.
- Requires well-aligned Validator to avoid false negatives or noise.
- Coordination complexity increases with number of iterations.

## Variations and Extensions
- **Multi-round Validator**: Uses temperature annealing or scoring thresholds to control the feedback loop.
- **Rotating Roles**: Worker and Validator swap roles periodically to simulate peer review.
- **Planner-Worker-Validator**: Validator ensures both planning and execution meet the goal.

## Example Use Cases
- Generate a product description from specifications.
- Answer a factual question using embedded knowledge.
- Write a function to perform a specific operation.

---

## Implementation

### Environment setup

This code prepares the notebook to run by loading configuration variables from a `.env` file in the project root.  
- The `.env` file contains private credentials such as the `OPENAI_API_KEY` required to authenticate with the OpenAI API.  
- It also specifies the default model to use (e.g., `CONF_OPENAI_DEFAULT_MODEL=openai/gpt-4o-mini`).  

We use `python-dotenv` to automatically load these values into the environment at runtime. This keeps secrets out of the codebase and makes it easy to switch models or keys without modifying the notebook.  

**Key points**  
- Make sure `.env` exists before running the notebook.  
- At minimum, define `OPENAI_API_KEY=sk-proj-XXXX`.  
- Optionally, set `CONF_OPENAI_DEFAULT_MODEL` to specify which model Worker and Validator agents will default to.  

In [ ]:
# Load the environment variables
from dotenv import load_dotenv
import os

load_dotenv(override=True)

agent_model_DEFAULT = os.getenv('CONF_OPENAI_DEFAULT_MODEL')

### Agent configuration file

The next two cells first create a temporary working directory and then write an `agents-config.yaml` file that specifies the Worker and Validator agents used in the pattern. By defining agents declaratively in YAML rather than embedding them directly in Python, the configuration becomes easier to read, test, and adjust. This separation also makes experiments reproducible, since the roles and their instructions can be version-controlled and swapped without changing the implementation code.

In the Worker–Validator pattern, this configuration provides a clear contract for how each agent should behave. The Worker is tuned to focus on execution, producing a direct output without unnecessary deliberation. The Validator is constrained to assessing whether the Worker’s response meets the task requirements and providing targeted feedback. Distinct settings for each role make the loop both flexible and controlled. For instance, the Worker can be given a slightly higher temperature to allow for more exploration, while the Validator runs with a lower temperature for stable, consistent evaluation.

**Key fields in YAML**  
- `name` – human-readable label for logs and traces  
- `instructions` – role prompt, with the Worker constrained to execution and the Validator to judgement  
- `model` – the model id for each role (they may differ)  
- `has_memory` – whether the agent keeps state across turns (set to False for the Worker here)  
- `temperature` – balance of exploration vs determinism  
- `max_tokens` – cap on response length  
- `output_type` – expected schema or type of output (e.g., `ValidatorResponse`)  

The design choice to externalize configuration improves portability and auditability but introduces a dependency on external files and the need for careful formatting. YAML’s indentation rules in particular can cause issues if not handled precisely.  

**Gotchas and extensions**  
- Ensure the model ids exist in your runtime/provider and that your API key has access  
- Make sure that the class corresponding to `output_type` is defined in the code and visible at the global level.

In [ ]:
!mkdir -p tmp

In [ ]:
%%writefile ./tmp/agents-config.yaml
# agents-config.yaml
worker:
  name: Basic Worker
  instructions: |
    You are a single-step Worker. Execute the given task directly and return the best possible result.
    Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
    Prefer concise, correct outputs. Use Markdown formatting when helpful.
  model: openai/gpt-4.1-nano
  has_memory: False
  temperature: 0.4
  max_tokens: 1000

validator:
  name: Basic Validator
  instructions: |
    You are a Validator agent in a Worker–Validator loop. Your role is to **assess whether 
    the Worker’s output meets the task objective and success criteria**.
    
    Your job is **not to redo the research yourself**, but to:
    1. Decide if the output is acceptable given the task and whatever success criteria is provided.
    2. If not, give precise, actionable feedback so the Worker can correct it.
  model: openai/gpt-4o-mini
  temperature: 0.2
  max_tokens: 1000
  output_type: ValidatorResponse

### ValidatorResponse class

This cell defines the `ValidatorResponse` class, a Pydantic model that enforces structure on the Validator’s output. Instead of returning free-form text, the Validator must respond in a consistent schema that can be parsed and acted upon. This ensures the feedback loop between Worker and Validator is machine-checkable, making it possible to automate approval, rejection, and revision cycles.

The schema captures three elements:  
- `status` signals whether the Worker’s output is approved or needs revision.  
- `rationale` provides a concise justification tied to the task’s success criteria.  
- `feedback` is optional and included only when revisions are needed, giving the Worker actionable instructions.  

This design keeps Validator outputs predictable and reduces the chance of misinterpretation. The `is_valid` method provides a quick way to check whether the output has been approved.

In [ ]:
# Define a structured output type for Validator

from pydantic import BaseModel, Field
from typing import Literal, Optional

class ValidatorResponse(BaseModel):
    status: Literal["approved", "needs_revision"]  = Field(..., description='Must be either "approved" or "needs_revision"')
    rationale: str = Field(..., description='Summarise your evaluation in 10 words, referencing specific success criteria.')
    feedback: Optional[str] = Field(None, description='Only include if status is "needs_revision". Provide concrete, actionable instructions for the Worker to correct or improve the output. Be explicit about which facts, quotes, or sections fail to meet audit-level verification standards.')

    def is_valid(self) -> bool:
        return True if self.status == "approved" else False


### Load and verify agent configuration

This cell reads the `agents-config.yaml` file created earlier and converts it into a Python dictionary. The configuration is then printed as formatted JSON so the notebook user can confirm it was parsed correctly and matches expectations. This step is primarily a sanity check: it ensures that both the Worker and Validator agents are defined as intended before they are instantiated. If the YAML has formatting issues or the fields are mis-specified, this is the point where they can be caught and corrected.  

In [ ]:
# Load the agents-config.yaml file

import json
import yaml

with open('./tmp/agents-config.yaml', 'r') as file:
    agent_config_data = yaml.safe_load(file)

formatted_json = json.dumps(agent_config_data, indent=4)
print(formatted_json)

### Create agents from configuration

This cell defines a helper function, `create_agent`, which builds an agent directly from the specifications in `agents-config.yaml`. The function generalizes agent creation so that any role defined in the configuration file—Worker, Validator, or future additions—can be instantiated in the same way.

The function first validates that the requested `agent_type` exists in the YAML. It then constructs a unique agent name with a timestamp, applies the role’s instructions and model, and sets model parameters such as temperature and maximum tokens. If the configuration specifies an `output_type`, the function maps it to a class defined in the notebook (e.g., `ValidatorResponse`). If the agent is configured to use memory, it creates a `SQLiteSession` to persist state across iterations.

A timestamp string (`now_string`) is used to generate unique identifiers for agent names and, where applicable, memory sessions. This prevents collisions when multiple agents of the same type are created, and makes log traces easier to differentiate. More generally, the function is designed to fall back to sensible defaults—such as using a timestamp-based name or the default model—when values are not explicitly provided in the YAML, balancing flexibility with reliability.

At the end of the cell, the function is used to instantiate both the Worker and Validator agents. This step effectively activates the configuration defined earlier, giving us two ready-to-use agents aligned to their roles in the Worker–Validator loop.

#### Note on Session-based memory in the Worker-Validator pattern

Although the function supports creating session-based memory through `SQLiteSession`, neither the Worker nor the Validator in this pattern require it. The Worker operates in a stateless manner, producing outputs solely from the given task, while the Validator only needs to judge the latest result against the original instructions. 

Memory is included here as a configurable option because the same mechanism may be useful in other patterns—for example, when agents need to track progress across multiple subtasks or carry forward context. Our broader intent is to keep agent configuration decoupled from the code that creates and executes them, so this framework can be reused and extended as additional agentic patterns are developed.

In [ ]:
from datetime import datetime
from agents import Agent, ModelSettings, SQLiteSession

def create_agent(agent_type: str = None):
    """ 
    Creates and returns an Agent that matches the given definition in the agents-config 
    YAML file. Optionally returns a memory Session if agent configuration calls for it.
    """

    if agent_type is None or not agent_type.strip():
        raise ValueError("agent_type must be a valid type of agent defined in agent-configs.yaml.")
    
    agent_config = agent_config_data.get(agent_type)
    if agent_config is None:
        raise ValueError(f"'{agent_type}' does not match an agent defined in agent-configs.yaml.")

    # Generate a timestamp string for unique naming
    now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

    # Build agent based on YAML specification
    try:
        agent_model_settings=ModelSettings(
            temperature=agent_config.get('temperature'),
            max_tokens=agent_config['max_tokens'],
        )

        agent_name = agent_config.get('name') or f"{agent_type}_{now_string}"
        
        new_agent = Agent(
            name=agent_name,
            instructions=agent_config['instructions'],
            model=agent_config.get('model') or agent_model_DEFAULT,
            output_type=globals().get(agent_config.get('output_type') or None),
            model_settings=agent_model_settings
        )
    except:
        raise

    # Create memory session for agent if configured
    agent_has_memory = agent_config.get('has_memory') or False
    agent_session_name = f"{agent_name}__SESSION_{now_string}" if agent_has_memory else None
    agent_session = SQLiteSession(agent_session_name) if agent_session_name else None

    return (new_agent, agent_session)

worker, worker_sess = create_agent('worker')
validator, validator_sess = create_agent('validator')


### Worker–Validator loop

This cell implements the control loop that coordinates the Worker and Validator agents. The function `assign_task` accepts a task description, optional success criteria, and a maximum number of iterations. It then orchestrates the dialogue: the Worker attempts the task, the Validator evaluates the output, and if the Validator requests revisions then the Worker receives feedback and tries again. The loop continues until the Validator approves the output or the iteration limit is reached.

The function constructs an interaction history that records the task, Worker outputs, and Validator assessments. This history is passed back into each subsequent round, giving both agents visibility into what has already been attempted. The `trace` context manager is used to record the interaction, which helps with debugging or later analysis.

Within the loop, the Worker produces an initial response which is stored in history. The Validator then runs against the same history and returns a structured `ValidatorResponse`. If the Validator approves the result, the function returns the Worker’s final output. Otherwise, the Validator’s feedback is appended to history, and the Worker is asked to try again.

A `max_loops` safeguard ensures that the process cannot run indefinitely if the Worker fails to satisfy the Validator. This provides a balance between quality control and computational cost, keeping the loop bounded even when success criteria are difficult to meet.

In [ ]:
# Dialog loop for a Worker agent and a Validoator agent. 

from agents import Runner, trace

async def assign_task(task: str, success_criteria: str = None, max_loops: int = 5) -> str:
    """
    Asssign a task to the Worker-Validator pair and receive their response in return.
    """

    # Check that Worker agent exists
    if worker is None:
        return "Worker agent has not been created."
    if validator is None:
        return "Validator agent has not been created."

    assignment = f"\ntask: {task}\nsuccess_criteria: {success_criteria}"
    history = [{"role": "user", "content": assignment}]

    count_loop = 0
    output_validated = False
    result = None

    with trace(f"{worker.name}_{validator.name}"):
        while (not output_validated) and (count_loop < max_loops):
            count_loop+=1

            try:
                worker_output = await Runner.run(starting_agent=worker, input=history)
                history.append({"role": "assistant", "content": worker_output.final_output})

                validator_output = await Runner.run(starting_agent=validator, input=history)
                assessment = validator_output.final_output
                history.append({"role": "assistant", "content": assessment.model_dump_json()})

            except Exception as e:
                return f"Error: {e}"
        
            output_validated = assessment.is_valid()
            result = worker_output.final_output if output_validated else assessment.model_dump_json()

    return result 

### Example execution of the cell-based loop

This cell demonstrates the cell-based `assign_task` function in action, before the standalone class is introduced. A concrete task is given to the Worker–Validator pair: generate a three-sentence executive summary of Tesla’s Q1 2024 earnings, with strict requirements around length, revenue detail, and comparison to Q4 2023. The accompanying success criteria serve as the Validator’s rubric, defining exactly what counts as an acceptable response.

By running this example, you can verify that the Worker produces an output, the Validator checks it against the criteria, and—if needed—the loop continues until the result is approved or the maximum iteration count is reached. This provides a working proof-of-concept that the Worker–Validator loop functions correctly in practice before moving on to the more modular class-based implementation.

In [ ]:
task = """ 
Write a three-sentence executive summary of the financial performance of Tesla's Q1 2024 earnings. Your summary must focus on revenue, and it must explicitly state whether the Q1 revenue was higher or lower than the Q4 2023 revenue.
"""

criteria = """ 
The output must be a summary of Tesla's Q1 2024 earnings.
The summary must be exactly three sentences long.
The summary must explicitly mention the exact Q1 2024 revenue figure in USD.
The summary must contain a clear statement comparing the Q1 2024 revenue to the Q4 2023 revenue (i.e., whether it was higher or lower).
The summary must not contain the word "profit" or "loss".
All factual data, including the revenue numbers and the comparison, must be accurate and verifiable from official sources.
"""

await assign_task(task, criteria)

### Standalone class implementation

This cell re-implements the Worker–Validator pattern as a standalone class that could be moved into a `.py` source file. Instead of scattering responsibilities across multiple notebook cells, the class bundles agent creation, task assignment, and history management into a single cohesive interface. This makes the pattern reusable, testable, and easier to extend in future work.

Several modifications are worth noting: 
- The class introduces a bounded `deque` to manage a rolling history of past interactions, allowing previous runs to be retrieved while preventing unbounded growth. 
- Agent creation is encapsulated within private methods, ensuring each role is consistently instantiated from the YAML configuration. 
- Unlike the earlier loop, the validator output here is parsed using `final_output_as` with a strict type check against `ValidatorResponse`. This enforces schema compliance and fails early if the Validator returns malformed output. 
- The `assign_task` method now returns a tuple containing both the result and a Boolean flag indicating whether validation succeeded, which provides clearer signalling to downstream code.

Another change is that session-based memory is not implemented, even if the YAML specifies it. The Worker–Validator pattern does not require agents to retain state across tasks, since the Worker focuses only on the current assignment and the Validator evaluates that single attempt. By omitting memory here, the class avoids unnecessary complexity while leaving the configuration mechanism in place for patterns that may need it.

Overall, this refactor preserves the same logic as the notebook cells but presents it in a form that is better suited for modular development. It illustrates the direction of treating these agentic patterns not as isolated experiments but as building blocks for a broader framework.

In [ ]:
# Class that implements the Worker-Validator pattern

import json
import os
import yaml
from collections import deque
from datetime import datetime
from agents import Agent, ModelSettings
from agents import Runner, trace
from pydantic import BaseModel, Field
from typing import Literal, Optional

class ValidatorResponse(BaseModel):
    """
    Represents the structured response from a validator agent.

    This Pydantic model ensures the validator's output is consistent and machine-readable.
    It includes a status indicating approval or a need for revision, a brief rationale,
    and detailed feedback if the output requires changes.
    """

    status: Literal["approved", "needs_revision"]  = Field(..., description='Must be either "approved" or "needs_revision"')
    rationale: str = Field(..., description='Summarise your evaluation in 10 words, referencing specific success criteria.')
    feedback: Optional[str] = Field(None, description='Only include if status is "needs_revision". Provide concrete, actionable instructions for the Worker to correct or improve the output. Be explicit about which facts, quotes, or sections fail to meet audit-level verification standards.')

    def is_valid(self) -> bool:
        return True if self.status == "approved" else False

class WorkerValidatorPattern:
    """
    A class that implements the Worker-Validator pattern for AI agent collaboration.

    This pattern orchestrates a loop where a 'worker' agent performs a task and a 'validator'
    agent evaluates the worker's output against a set of success criteria. The validator provides
    feedback to the worker if the output needs revision. The process repeats until the
    validator approves the output or a maximum number of loops is reached.

    The class uses a YAML configuration file to define and initialize the worker and validator
    agents. It maintains a history of the conversation, allowing agents to have context
    from previous turns.
    """

    def __init__(self, agent_config_filename: str = './tmp/agents-config.yaml') -> None:
        """
        Initialize the pattern from the agents-config.yaml file
        """

        self.__agent_model_DEFAULT = os.getenv('CONF_OPENAI_DEFAULT_MODEL')

        # Load the agents-config.yaml file
        with open(agent_config_filename, 'r') as file:
            self.__agent_config_data = yaml.safe_load(file)
        
        # Create an empty history log
        self.__max_history = 5
        self.__history_log = deque(maxlen=self.__max_history)

        # Create the agents for this pattern
        self.__init_agents() 


    def __new_history(self) -> dict:
        """
        Add a new empty conversation history to the log and return it.
        """

        history = []
        self.__history_log.append(history)

        return history
    
    def get_history(self, age: int = 0) -> Optional[dict]:
        """
        Retrieves a stored history dictionary, using a zero-based index for age (0 is most recent).
        """
        if not self.__history_log:
            return None
        
        index = -1 * min((age + 1), len(self.__history_log))
        try:
            return self.__history_log[index]
        except IndexError:
            return None
        
    
    def __create_agent(self, agent_type: str = None) -> Agent:
        """
        Creates and returns an Agent that matches the given definition in the agents-config YAML file.
        """

        if agent_type is None or not agent_type.strip():
            raise ValueError("agent_type must be a valid type of agent defined in agent-configs.yaml.")
        
        agent_config = self.__agent_config_data.get(agent_type)
        if agent_config is None:
            raise ValueError(f"'{agent_type}' does not match an agent defined in agent-configs.yaml.")

        # Generate a timestamp string for unique naming
        now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

        # Build agent based on YAML specification
        agent_model_settings=ModelSettings(
            temperature=agent_config.get('temperature'),
            max_tokens=agent_config.get('max_tokens')
        )
        
        new_agent = Agent(
            name=agent_config.get('name') or f"{agent_type}_{now_string}",
            instructions=agent_config.get('instructions'),
            model=agent_config.get('model') or self.__agent_model_DEFAULT,
            output_type=globals().get(agent_config.get('output_type')),
            model_settings=agent_model_settings
        )

        return new_agent
    
    
    def __init_agents(self) -> int:
        """
        Creates an instance of each of the agent types loaded from agents-config.yaml.
        Returns the number of agents created.
        """
        
        self.__agents = {}

        for agent_type in self.__agent_config_data:
            new_agent = self.__create_agent(agent_type=agent_type)
        
            if new_agent is not None:
                self.__agents[agent_type] = new_agent

        return len(self.__agents)
    

    async def assign_task(self, 
                          task: str, 
                          success_criteria: str = None, 
                          max_loops: int = 5) -> tuple[str, bool]:
        """
        Assigns a task to a Worker-Validator pair and awaits a final response.

        Args:
            task (str): The primary task for the agents to execute.
            success_criteria (str, optional): Describes the criteria to be used in validating the output.
                                              Defaults to None.
            max_loops (int, optional): The maximum number of validation loops to attempt.
                                       Defaults to 5.

        Returns:
            tuple[str, bool]: A tuple containing:
                - str: The final, validated output from the Worker agent, or validator's failure 
                       assessment if output was not validated after maximum turns.
                - bool: A flag indicating if the output was successfully validated.
        """

        # Obtain worker and validator agents
        worker = self.__agents.get("worker")
        validator = self.__agents.get("validator")
 
        if worker is None or validator is None:
            raise NameError("One or more agents were not created. Are they correctly defined in agent-configs.yaml?")

        # Seed a new history dictionary
        history = self.__new_history()
        assignment = f"task: {task}"
        if success_criteria is not None and len(success_criteria) > 0:
            assignment += f"\nsuccess_criteria: {success_criteria}"
        history.append({"role": "user", "content": assignment})

        # Loop between worker completing the task and validator checking it. Exit when validator approves
        # or the number of iterations reaches max_loops.
        count_loop = 0
        output_validated = False
        result = None

        with trace(f"{worker.name}_{validator.name}"):
            while (not output_validated) and (count_loop < max_loops):
                count_loop+=1

                try:
                    worker_output = await Runner.run(starting_agent=worker, input=history)
                    history.append({"role": "assistant", "content": worker_output.final_output})
                except Exception as e:
                    history.append({"role": "user", "content": f"Error: {e}"})

                try:
                    validator_output = await Runner.run(starting_agent=validator, input=history)
                    assessment = validator_output.final_output_as(ValidatorResponse, raise_if_incorrect_type=True)
                    history.append({"role": "assistant", "content": assessment.model_dump_json()})

                    output_validated = assessment.is_valid()
                    result = worker_output.final_output if output_validated else assessment.model_dump_json()
                except Exception as e:
                    history.append({"role": "user", "content": f"Error: {e}"})
                    output_validated = False
                    result = f"Error: {e}"
            
        return (result, output_validated)
    

### Gradio demo interface

The final cell provides a simple Gradio interface for trying out the Worker–Validator pattern interactively. The UI lets you enter a task, optional success criteria, and a maximum number of validation loops. When you click **Run Task**, the Worker attempts the task, the Validator reviews it, and the result is displayed.

The `use_pattern` flag determines which implementation is used:
- If set to `True`, the demo runs with the class-based `WorkerValidatorPattern`. This version also displays the recorded interaction history, making it easier to see how the Worker and Validator collaborated.  
- If set to `False`, the demo uses the earlier notebook functions. This mode is simpler but does not track history.  

In addition to the Gradio interface, execution can be monitored through [OpenAI traces](https://platform.openai.com/traces). Each run within the loop is wrapped with a `trace` context, allowing you to inspect the flow of messages between Worker and Validator, debug errors, and analyse performance in the OpenAI dashboard. This provides visibility into how the pattern operates under the hood and helps refine prompts, parameters, and loop design.  

By wrapping the pattern in a Gradio app, this cell turns the Worker–Validator design from an abstract loop into a tangible demo. It is especially useful for experimenting with different tasks, tweaking success criteria, and observing how many iterations the loop requires to reach an approved result.

In [ ]:
# Create a gradio interface to interact with the Worker-Validator pattern

import gradio as gr

use_pattern = True

with gr.Blocks(title="Worker-Validator — Agentic Design Patterns") as demo:

    pattern = WorkerValidatorPattern(agent_config_filename='./tmp/agents-config.yaml') if use_pattern else None

    gr.Markdown("""
    # 🛠️ Worker-Validator Pattern
    
    This is a simple worker-validator that can be used to perform single-step execution tasks whose output is validated before being returned.
    """)

    with gr.Row():
        with gr.Column(scale=9):
            task = gr.Textbox(label="Task", placeholder="What task do you want completed?", lines=4)
            success_criteria = gr.Textbox(label="Success Criteria", placeholder="Describe what success looks like.", lines=4)
        with gr.Column(scale=1):
            max_loops = gr.Number(label="Maximum loops", value=3)
            run_btn = gr.Button("Run Task", variant="primary")
    out = gr.Markdown(label="Result", show_label=True)
    history = gr.JSON(label="History", show_label=True)

    async def on_run(task:str, success_criteria: str, max_loops: int):
        run_btn.interactive = False  # disable button while running
        try:
            output = None
            hist = None
            if use_pattern:
                output, _ = await pattern.assign_task(task=task, success_criteria=success_criteria, max_loops=max_loops)
                hist_dict = pattern.get_history()
                hist = json.dumps(hist_dict, indent=2)
            else:
                output = await assign_task(task, success_criteria, max_loops)
                hist = "{}"
            return (output, hist)
        finally:
            run_btn.interactive = True  # re-enable button

    run_btn.click(on_run, inputs=[task, success_criteria, max_loops], outputs=[out, history])
    task.submit(on_run, inputs=[task, success_criteria, max_loops], outputs=[out, history])

demo.launch(inline=True)